In [1]:
import os,shutil,random,re
import numpy as np
import time
import json
import cv2
from matplotlib import pyplot as plt
from PIL import Image
np.set_printoptions(suppress=True)

In [2]:
def check_size_all_from_path(data_dir, dx = 1280, dy = 720):
    '''
    Check the size of the picture according to the path
    # Input: Path, expected picture size
    # Output: For each category, the total quantity, and the quantity that does not meet the requirements
    '''
    poses = [] # Store the names of the pictures for all categories.
    cates = os.listdir(data_dir) # How many categories under this path
    print(f'# classes {len(cates)}')
    nums_wrongsize = np.zeros(len(cates)) 
    nums = np.zeros(len(cates)) 
    for k in range(len(cates)): 
        poses.append(os.listdir(data_dir+'/'+cates[k]))  
        nums[k] = len(os.listdir(data_dir+'/'+cates[k])) 
    for k in range(len(cates)): # Check the number of unexpected pictures
        _t0 = time.time()
        for i in range(len(poses[k])):
            _img = Image.open(data_dir+'/'+cates[k]+'/'+poses[k][i])
            if _img.size[1] != dx or _img.size[0] != dy:
                nums_wrongsize[k] += 1
        _t1 = time.time(); print(f'Cate{k}: {np.round(_t1-_t0,2)}s')  
    print(cates,'\n',nums,'\n', nums_wrongsize)

In [3]:
data_dir = '/mnt/multi-class_simu/real data/Audi/data/Images_all_8cls'
check_size_all_from_path(data_dir, dx = 1280, dy = 720)

# classes 9
Cate0: 5.43s
Cate1: 17.99s
Cate2: 4.59s
Cate3: 3.04s
Cate4: 4.52s
Cate5: 0.0s
Cate6: 3.07s
Cate7: 4.54s
Cate8: 5.27s
['A6L', 'Q5L Sportback', 'A4L', 'A3 Sportback', 'RS5', '.ipynb_checkpoints', 'A7 Sportback', 'Q7', 'A5 Sportback'] 
 [347. 386. 340. 334. 307.   0. 241. 289. 315.] 
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
cate_order = ['0','A3 Sportback','A4L','A5 Sportback','A6L', 'A7 Sportback', 'Q5L Sportback', 'Q7', 'RS5']
cate = cate_order[1:]
m = np.load('/mnt/multi-class_simu/real data/Audi/data/master_file_8cls.npy').tolist()

In [55]:
def check_size_bbox_all(data_dir, master_file, dx = 1280, dy = 720):
    for i in range(len(master_file)):
        if i % 1000 == 0: _t0 = time.time()
        infor = master_file[i].split('\t')
        _name = infor[1]; _cate = infor[0]
        _box = re.split('[\[;,;\n;\]; ]',infor[-1])
        _ymin = int(_box[5]); _ymax = int(_box[7])
        _xmin = int(_box[1]); _xmax = int(_box[3]) 

        _img = Image.open(data_dir+_cate+'/'+_name)
        if _img.size[1] != dx or _img.size[0] != dy:
            print(infor)
        if i % 1000 == 0: _t1 = time.time(); print(f'pic{i}: {np.round(_t1-_t0,2)}s')  

In [57]:
data_dir = '/mnt/multi-class_simu/real data/Audi/data/Images_all_8cls/'
check_size_bbox_all(data_dir, m, dx = 1280, dy = 720)

pic0: 0.0s
pic1000: 0.0s
pic2000: 0.0s


In [6]:
# Split the train data and test data
np.random.seed(0)
idx_train = np.random.choice(np.arange(len(m)), int(0.8*len(m)), replace = False)
idx_train = np.sort(idx_train)
idx_test = np.setdiff1d(np.arange(len(m)), idx_train)

master_file_train = list(np.array(m)[idx_train])
master_file_test = list(np.array(m)[idx_test])